In [ ]:
# default_exp shopping

# Shopping
simulates shopping with python code. 
this is designed to test the backend and allow conversion to frontend

In [3]:
#export
from villaProductSdk.products import ProductSdk
from villaBackendSdk.basket import BasketSdk
from villaProductDatabase import database
from villaProductDatabase.schema import Response, Event
from s3bz.s3bz import S3
import json

import pandas as pd

missing environment variable 'INVENTORY_BUCKET_NAME' in Database s3 NB
missing environment variable 'INVENTORY_BUCKET_NAME' in query NB
missing environment variable 'INPUT_BUCKET_NAME' in update NB
error, missing environment variables 
'DATABASE_TABLE_NAME'
dax endpoint missing 'DAX_ENDPOINT'


## tests

### Search for items

In [4]:
from requests import post, get
def search(searchQuery:str):
  endpoint = 'https://8av9li9v82.execute-api.ap-southeast-1.amazonaws.com/Prod/orismaSearch/'
  queries = {
    'data_t3':searchQuery
  }
  response = get(endpoint, params=queries)
  return response.json()

In [5]:
%time
result = search('strawberry')
resultList = pd.DataFrame(result)['pr_code'].to_list()
resultList[:10]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


['0209667',
 '0178705',
 '0207825',
 '0208965',
 '0229543',
 '0229471',
 '0209997',
 '0212434',
 '0207999',
 '0197272']

In [6]:
sdk = ProductSdk(branch = 'dev-manual', user=None, pw=None)

In [7]:
sdk.querySingleProduct(resultList[0])

ERROR:root:error parsing body, perhaps there is no body in response      response is {'errorMessage': "'dict' object has no attribute 'data'", 'errorType': 'AttributeError', 'stackTrace': ['  File "/var/task/villaProductDatabase/database.py", line 167, in lambdaSingleQuery\n    result = ProductDatabase.singleProductQuery({key:value}).data\n']}
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python38/lib/python3.8/site-packages/villaProductSdk/products.py", line 47, in returnLambdaResponse
    return Response.fromDict(lambdaResponse).body
  File "/home/ec2-user/anaconda3/envs/python38/lib/python3.8/site-packages/villaProductSdk/schema.py", line 27, in fromDict
    body = dictInput.pop('body')
KeyError: 'body'
ERROR:root:{'errorMessage': "'dict' object has no attribute 'data'", 'errorType': 'AttributeError', 'stackTrace': ['  File "/var/task/villaProductDatabase/database.py", line 167, in lambdaSingleQuery\n    result = ProductDatabase.singleProductQuery({key:val

In [9]:
sdk.querySingleProduct('0117939')

{'cprcode': '0117939',
 'iprcode': '0117939',
 'oprcode': '0117939',
 'ordertype': 'Y',
 'pr_abb': 'COLES TRADITIONAL CH',
 'pr_active': 'Y',
 'pr_cgcode': '98',
 'pr_code': '0117939',
 'pr_dpcode': '28',
 'pr_engname': 'COLES TRADITIONAL CHRISTMAS PUDDING 454G',
 'pr_ggcode': '002',
 'pr_market': 'COLES TRADITIONAL CHRISTMAS PU',
 'pr_name': 'COLES TRADITIONAL CHRISTMAS PUDDING 454G.',
 'pr_puqty': '6.00',
 'pr_sa_method': '1',
 'pr_sucode1': 'F1239',
 'pr_suref3': 'S',
 'prtype': 'I',
 'psqty': '1',
 'pstype': '1',
 'pr_country_th': '',
 'pr_country_en': 'United Kingdom',
 'pr_keyword_th': '',
 'pr_keyword_en': '',
 'pr_filter_th': 'Seasonal,Crisps  Snacks,Grocery',
 'pr_filter_en': 'Seasonal,Crisps  Snacks,Grocery',
 'online_category_l1_th': 'สินค้าเทศกาล',
 'online_category_l1_en': 'Festive Items',
 'online_category_l2_th': 'สินค้าคริสต์มาสและปีใหม่',
 'online_category_l2_en': 'Christmas & New Year',
 'online_category_l3_th': '',
 'online_category_l3_en': '',
 'villa_category_l1_en